# importing libraries

In [1]:
import pandas as pd
import numpy as np
import pymysql
import os

# importing csv file

In [2]:
df=pd.read_csv(r"C:\BOSS\project\zepto_v1.csv")

In [3]:
df.head(10)

,Category,name,mrp,discountPercent,availableQuantity,discountedSellingPrice,weightInGms,outOfStock,quantity
0,Fruits & Vegetables,Onion,2500,16,3,2100,1000,False,1
1,Fruits & Vegetables,Tomato Hybrid,4200,16,3,3500,1000,False,1
2,Fruits & Vegetables,Tender Coconut,5100,15,3,4300,58,False,1
3,Fruits & Vegetables,Coriander Leaves,2000,15,3,1700,100,False,100
4,Fruits & Vegetables,Ladies Finger,1400,14,3,1200,250,False,250
5,Fruits & Vegetables,Potato,3500,17,3,2900,1000,False,1
6,Fruits & Vegetables,Lemon,7500,16,3,6300,200,False,200
7,Fruits & Vegetables,Watermelon,5800,15,3,4900,58,False,1
8,Fruits & Vegetables,Capsicum Green,2300,17,3,1900,250,False,250
9,Fruits & Vegetables,Chilli Green,1900,15,3,1600,100,False,100


In [4]:
# columns and its header in the table

# exploring the dataset

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3732 entries, 0 to 3731
Data columns (total 9 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Category                3732 non-null   object
 1   name                    3732 non-null   object
 2   mrp                     3732 non-null   int64 
 3   discountPercent         3732 non-null   int64 
 4   availableQuantity       3732 non-null   int64 
 5   discountedSellingPrice  3732 non-null   int64 
 6   weightInGms             3732 non-null   int64 
 7   outOfStock              3732 non-null   bool  
 8   quantity                3732 non-null   int64 
dtypes: bool(1), int64(6), object(2)
memory usage: 237.0+ KB


In [6]:
# this gets inforamtion like column name ,data types of the data

In [7]:
# here both mrp and discounted selling price bth are in int datatype and need to convert into flot datatypes beacuse it is the better daata type of price

In [8]:
df[['mrp','discountedSellingPrice']]=df[['mrp','discountedSellingPrice']].astype(float)

# checking and removing null

In [9]:
df.isnull().sum()

Category                  0
name                      0
mrp                       0
discountPercent           0
availableQuantity         0
discountedSellingPrice    0
weightInGms               0
outOfStock                0
quantity                  0
dtype: int64

In [10]:
 #here we an see that there is no null values in the table

# checking and removing duplicates

In [11]:
df.duplicated().sum()

2

In [12]:
#here we have 2 duplicate rows  present in the datset

In [13]:
df[df.duplicated(keep=False)]

,Category,name,mrp,discountPercent,availableQuantity,discountedSellingPrice,weightInGms,outOfStock,quantity
2908,Personal Care,Listerine Cool Mint Mouthwash - Mild Taste,15000.0,10,6,13500.0,250,False,250
2946,Personal Care,Listerine Cool Mint Mouthwash - Mild Taste,15000.0,10,6,13500.0,250,False,250
3252,Paan Corner,Listerine Cool Mint Mouthwash - Mild Taste,15000.0,10,6,13500.0,250,False,250
3290,Paan Corner,Listerine Cool Mint Mouthwash - Mild Taste,15000.0,10,6,13500.0,250,False,250


In [14]:
df.drop_duplicates(inplace=True)
df.duplicated().sum()

0

In [15]:
#removed duplicates from the  table and we have 0 duplicates

In [16]:
import os
import pandas as pd
import pymysql


csv_files = [
    ('zepto_v1.csv', 'zepto')
]


folder_path = r'C:\BOSS\project'


conn = pymysql.connect(
    host='localhost',
    user='root',
    password='Ganesh@2000',
    database='zepto'
)
cursor = conn.cursor()


def get_sql_type(dtype):
    if pd.api.types.is_integer_dtype(dtype):
        return 'INT'
    elif pd.api.types.is_float_dtype(dtype):
        return 'FLOAT'
    elif pd.api.types.is_bool_dtype(dtype):
        return 'BOOLEAN'
    elif pd.api.types.is_datetime64_any_dtype(dtype):
        return 'DATETIME'
    else:
        return 'TEXT'


for csv_file, table_name in csv_files:
    file_path = os.path.join(folder_path, csv_file)
    
    df = pd.read_csv(file_path)
    df = df.where(pd.notnull(df), None)

    print(f"Processing {csv_file}")
    print(f"NaN values before replacement:\n{df.isnull().sum()}\n")

    df.columns = [col.replace(' ', '_').replace('-', '_').replace('.', '_') for col in df.columns]

    columns = ', '.join([f'`{col}` {get_sql_type(df[col].dtype)}' for col in df.columns])
    create_table_query = f'CREATE TABLE IF NOT EXISTS `{table_name}` ({columns})'
    cursor.execute(create_table_query)

    for _, row in df.iterrows():
        values = tuple(None if pd.isna(x) else x for x in row)
        sql = f"INSERT INTO `{table_name}` ({', '.join(['`' + col + '`' for col in df.columns])}) VALUES ({', '.join(['%s'] * len(row))})"
        cursor.execute(sql, values)

    conn.commit()

conn.close()

Processing zepto_v1.csv
NaN values before replacement:
Category                  0
name                      0
mrp                       0
discountPercent           0
availableQuantity         0
discountedSellingPrice    0
weightInGms               0
outOfStock                0
quantity                  0
dtype: int64

